In [1]:
import tensorflow as tf

In [14]:
#5.1

In [2]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256,activation=tf.nn.relu),
    tf.keras.layers.Dense(10),
])

In [3]:
X = tf.random.uniform((2,20))
net(X)

2021-10-27 10:06:23.369044: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-10-27 10:06:23.412097: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:17:00.0 name: GeForce RTX 3090 computeCapability: 8.6
coreClock: 1.785GHz coreCount: 82 deviceMemorySize: 23.70GiB deviceMemoryBandwidth: 871.81GiB/s
2021-10-27 10:06:23.413586: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 1 with properties: 
pciBusID: 0000:65:00.0 name: GeForce RTX 3090 computeCapability: 8.6
coreClock: 1.785GHz coreCount: 82 deviceMemorySize: 23.70GiB deviceMemoryBandwidth: 871.81GiB/s
2021-10-27 10:06:23.413747: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2021-10-27 10:06:23.415230: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
202

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.42025387,  0.5844606 ,  0.4185487 , -0.03543741,  0.30877548,
        -0.21616144,  0.23725063, -0.30998796, -0.03085215,  0.36529422],
       [ 0.30915403,  0.09992593,  0.4133116 , -0.16274762, -0.12876947,
        -0.15452962,  0.09960232, -0.06967045, -0.04102004, -0.02586354]],
      dtype=float32)>

In [4]:
class MLP(tf.keras.Model):
    # 用模型参数声明层。这里，我们声明两个全连接的层
    def __init__(self):
        # 调用`MLP`的父类`Block`的构造函数来执行必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数`params`（稍后将介绍）
        super().__init__()
        # Hidden layer
        self.hidden = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)
        self.out = tf.keras.layers.Dense(units=10)  # Output layer

    # 定义模型的正向传播，即如何根据输入`X`返回所需的模型输出
    def call(self, X):
        return self.out(self.hidden((X)))

In [5]:
net = MLP()
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.01892718,  0.03442106, -0.03709103, -0.25153875,  0.1339964 ,
         0.04853793,  0.12294535,  0.03671088, -0.22283463, -0.20887223],
       [ 0.14022581,  0.40567994, -0.19877303, -0.1979876 , -0.05517552,
         0.04119499,  0.12078217,  0.04348309, -0.18448439, -0.4255119 ]],
      dtype=float32)>

In [7]:
net.summary()

Model: "mlp"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              multiple                  5376      
_________________________________________________________________
dense_3 (Dense)              multiple                  2570      
Total params: 7,946
Trainable params: 7,946
Non-trainable params: 0
_________________________________________________________________


In [8]:
class MySequential(tf.keras.Model):
    def __init__(self, *args):
        super().__init__()
        self.modules = []
        for block in args:
            # 这里，`block`是`tf.keras.layers.Layer`子类的一个实例。
            self.modules.append(block)

    def call(self, X):
        for module in self.modules:
            X = module(X)
        return X

In [9]:
net = MySequential(
    tf.keras.layers.Dense(units=256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10))
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.13596225, -0.42076984,  0.4828717 , -0.11863811, -0.25491825,
         0.17139563,  0.00105003,  0.02972568, -0.01912697,  0.25519523],
       [ 0.43733844, -0.43543112,  0.29148492,  0.01902463, -0.47865397,
         0.21552086,  0.14211564, -0.04099028, -0.11882308,  0.28752103]],
      dtype=float32)>

In [10]:
class FixedHiddenMLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        # 使用`tf.constant`函数创建的随机权重参数在训练期间不会更新（即为常量参数）。
        self.rand_weight = tf.constant(tf.random.uniform((20, 20)))
        self.dense = tf.keras.layers.Dense(20, activation=tf.nn.relu)

    def call(self, inputs):
        X = self.flatten(inputs)
        # 使用创建的常量参数以及`relu`和`matmul`函数。
        X = tf.nn.relu(tf.matmul(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数。
        X = self.dense(X)
        # 控制流
        while tf.reduce_sum(tf.math.abs(X)) > 1:
            X /= 2
        return tf.reduce_sum(X)

In [11]:
net = FixedHiddenMLP()
net(X)

<tf.Tensor: shape=(), dtype=float32, numpy=0.7149376>

In [12]:
class NestMLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.net = tf.keras.Sequential()
        self.net.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))
        self.net.add(tf.keras.layers.Dense(32, activation=tf.nn.relu))
        self.dense = tf.keras.layers.Dense(16, activation=tf.nn.relu)

    def call(self, inputs):
        return self.dense(self.net(inputs))

chimera = tf.keras.Sequential()
chimera.add(NestMLP())
chimera.add(tf.keras.layers.Dense(20))
chimera.add(FixedHiddenMLP())
chimera(X)

<tf.Tensor: shape=(), dtype=float32, numpy=0.5126094>

In [16]:
net.summary()

Model: "fixed_hidden_mlp"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            multiple                  0         
_________________________________________________________________
dense_6 (Dense)              multiple                  420       
Total params: 420
Trainable params: 420
Non-trainable params: 0
_________________________________________________________________


In [13]:
#5.2

In [17]:
import tensorflow as tf

net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4, activation=tf.nn.relu),
    tf.keras.layers.Dense(1),
])

X = tf.random.uniform((2, 4))
net(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[-1.6989639],
       [-1.1149968]], dtype=float32)>

In [18]:
print(net.layers[2].weights)

[<tf.Variable 'sequential_3/dense_13/kernel:0' shape=(4, 1) dtype=float32, numpy=
array([[-1.0553765 ],
       [-0.79944146],
       [-0.9164586 ],
       [-0.00261855]], dtype=float32)>, <tf.Variable 'sequential_3/dense_13/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]


In [19]:
print(type(net.layers[2].weights[1]))
print(net.layers[2].weights[1])
print(tf.convert_to_tensor(net.layers[2].weights[1]))

<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'>
<tf.Variable 'sequential_3/dense_13/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>
tf.Tensor([0.], shape=(1,), dtype=float32)


In [20]:
print(net.layers[1].weights)
print(net.get_weights())

[<tf.Variable 'sequential_3/dense_12/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[ 0.8524328 , -0.42713472, -0.7695273 ,  0.7103122 ],
       [ 0.2144013 ,  0.8100081 , -0.27583718,  0.76165444],
       [ 0.55537564, -0.1186536 ,  0.30644757,  0.3617522 ],
       [ 0.37697536,  0.00607699,  0.5920847 , -0.7321231 ]],
      dtype=float32)>, <tf.Variable 'sequential_3/dense_12/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>]
[array([[ 0.8524328 , -0.42713472, -0.7695273 ,  0.7103122 ],
       [ 0.2144013 ,  0.8100081 , -0.27583718,  0.76165444],
       [ 0.55537564, -0.1186536 ,  0.30644757,  0.3617522 ],
       [ 0.37697536,  0.00607699,  0.5920847 , -0.7321231 ]],
      dtype=float32), array([0., 0., 0., 0.], dtype=float32), array([[-1.0553765 ],
       [-0.79944146],
       [-0.9164586 ],
       [-0.00261855]], dtype=float32), array([0.], dtype=float32)]


In [21]:
net.get_weights()[1]

array([0., 0., 0., 0.], dtype=float32)

In [22]:
def block1(name):
    return tf.keras.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(4, activation=tf.nn.relu)],
        name=name)

def block2():
    net = tf.keras.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add(block1(name=f'block-{i}'))
    return net

rgnet = tf.keras.Sequential()
rgnet.add(block2())
rgnet.add(tf.keras.layers.Dense(1))
rgnet(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.],
       [0.]], dtype=float32)>

In [24]:
rgnet.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_5 (Sequential)    multiple                  80        
_________________________________________________________________
dense_18 (Dense)             multiple                  5         
Total params: 85
Trainable params: 85
Non-trainable params: 0
_________________________________________________________________


In [25]:
rgnet.layers[0].layers[1].layers[1].weights[1]

<tf.Variable 'sequential_4/sequential_5/block-1/dense_15/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>

In [26]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4, activation=tf.nn.relu,
        kernel_initializer=tf.random_normal_initializer(mean=0, stddev=0.01),
        bias_initializer=tf.zeros_initializer()),
    tf.keras.layers.Dense(1)])

net(X)
net.weights[0], net.weights[1]

(<tf.Variable 'sequential_6/dense_19/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[-0.02608244, -0.00242572,  0.00460067, -0.00565759],
        [ 0.00571476, -0.00078479, -0.01786315,  0.01032325],
        [ 0.01408292, -0.00457417,  0.00165934, -0.01139633],
        [ 0.01077343,  0.00188405, -0.00669253,  0.00925974]],
       dtype=float32)>,
 <tf.Variable 'sequential_6/dense_19/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>)

In [27]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4, activation=tf.nn.relu,
        kernel_initializer=tf.keras.initializers.Constant(1),
        bias_initializer=tf.zeros_initializer()),
    tf.keras.layers.Dense(1),
])

net(X)
net.weights[0], net.weights[1]

(<tf.Variable 'sequential_7/dense_21/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], dtype=float32)>,
 <tf.Variable 'sequential_7/dense_21/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>)

In [28]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4,
        activation=tf.nn.relu,
        kernel_initializer=tf.keras.initializers.GlorotUniform()),
    tf.keras.layers.Dense(
        1, kernel_initializer=tf.keras.initializers.Constant(1)),
])

net(X)
print(net.layers[1].weights[0])
print(net.layers[2].weights[0])

<tf.Variable 'sequential_8/dense_23/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[-0.4046015 ,  0.16566402, -0.6033228 ,  0.25067157],
       [ 0.4784636 , -0.36495614,  0.19036931,  0.4109885 ],
       [ 0.30405325,  0.08318692, -0.77716035, -0.25461942],
       [-0.32151514, -0.80836165,  0.39931577, -0.47657326]],
      dtype=float32)>
<tf.Variable 'sequential_8/dense_24/kernel:0' shape=(4, 1) dtype=float32, numpy=
array([[1.],
       [1.],
       [1.],
       [1.]], dtype=float32)>


In [29]:
class MyInit(tf.keras.initializers.Initializer):
    def __call__(self, shape, dtype=None):
        data=tf.random.uniform(shape, -10, 10, dtype=dtype)
        factor=(tf.abs(data) >= 5)
        factor=tf.cast(factor, tf.float32)
        return data * factor

net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4,
        activation=tf.nn.relu,
        kernel_initializer=MyInit()),
    tf.keras.layers.Dense(1),
])

net(X)
print(net.layers[1].weights[0])

<tf.Variable 'sequential_9/dense_25/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[-0.       , -5.2362466, -8.197484 , -7.2174335],
       [ 0.       , -6.728115 ,  0.       , -8.632565 ],
       [-0.       , -0.       , -0.       ,  6.0986805],
       [ 0.       , -6.94957  , -6.4328146,  0.       ]], dtype=float32)>


In [30]:
net.layers[1].weights[0][:].assign(net.layers[1].weights[0] + 1)
net.layers[1].weights[0][0, 0].assign(42)
net.layers[1].weights[0]

<tf.Variable 'sequential_9/dense_25/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[42.       , -4.2362466, -7.197484 , -6.2174335],
       [ 1.       , -5.728115 ,  1.       , -7.6325645],
       [ 1.       ,  1.       ,  1.       ,  7.0986805],
       [ 1.       , -5.94957  , -5.4328146,  1.       ]], dtype=float32)>

In [31]:
# tf.keras的表现有点不同。它会自动删除重复层
shared = tf.keras.layers.Dense(4, activation=tf.nn.relu)
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    shared,
    shared,
    tf.keras.layers.Dense(1),
])

net(X)
# 检查参数是否不同
print(len(net.layers) == 3)

True


In [32]:
#5.3

In [33]:
import tensorflow as tf

net = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10),
])

In [34]:
[net.layers[i].get_weights() for i in range(len(net.layers))]

[[], []]

In [35]:
X = tf.random.uniform((2, 20))
net(X)
[w.shape for w in net.get_weights()]

[(20, 256), (256,), (256, 10), (10,)]

In [36]:
#5.4

In [37]:
import tensorflow as tf


class CenteredLayer(tf.keras.Model):
    def __init__(self):
        super().__init__()

    def call(self, inputs):
        return inputs - tf.reduce_mean(inputs)

In [38]:
layer = CenteredLayer()
layer(tf.constant([1, 2, 3, 4, 5]))

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([-2, -1,  0,  1,  2], dtype=int32)>

In [39]:
net = tf.keras.Sequential([tf.keras.layers.Dense(128), CenteredLayer()])

In [40]:
Y = net(tf.random.uniform((4, 8)))
tf.reduce_mean(Y)

<tf.Tensor: shape=(), dtype=float32, numpy=3.7252903e-09>

In [41]:
class MyDense(tf.keras.Model):
    def __init__(self, units):
        super().__init__()
        self.units = units

    def build(self, X_shape):
        self.weight = self.add_weight(name='weight',
            shape=[X_shape[-1], self.units],
            initializer=tf.random_normal_initializer())
        self.bias = self.add_weight(
            name='bias', shape=[self.units],
            initializer=tf.zeros_initializer())

    def call(self, X):
        linear = tf.matmul(X, self.weight) + self.bias
        return tf.nn.relu(linear)

In [42]:
dense = MyDense(3)
dense(tf.random.uniform((2, 5)))
dense.get_weights()

[array([[ 0.0017721 ,  0.05793003, -0.03384516],
        [ 0.05414852,  0.05358556,  0.07671374],
        [-0.0361996 , -0.04363196,  0.04037216],
        [ 0.04978222,  0.03155075,  0.12934138],
        [ 0.08498754, -0.0268233 , -0.03434233]], dtype=float32),
 array([0., 0., 0.], dtype=float32)]

In [43]:
dense(tf.random.uniform((2, 5)))

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0.08682563, 0.00440838, 0.06056979],
       [0.05445284, 0.00501781, 0.02500721]], dtype=float32)>

In [44]:
net = tf.keras.models.Sequential([MyDense(8), MyDense(1)])
net(tf.random.uniform((2, 64)))

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.00783756],
       [0.01797368]], dtype=float32)>

In [45]:
#5.5

In [46]:
import numpy as np
import tensorflow as tf

x = tf.range(4)
np.save('x-file.npy', x)

In [47]:
x2 = np.load('x-file.npy', allow_pickle=True)
x2

array([0, 1, 2, 3], dtype=int32)

In [48]:
y = tf.zeros(4)
np.save('xy-files.npy', [x, y])
x2, y2 = np.load('xy-files.npy', allow_pickle=True)
(x2, y2)

(array([0., 1., 2., 3.]), array([0., 0., 0., 0.]))

In [49]:
mydict = {'x': x, 'y': y}
np.save('mydict.npy', mydict)
mydict2 = np.load('mydict.npy', allow_pickle=True)
mydict2

array({'x': <tf.Tensor: shape=(4,), dtype=int32, numpy=array([0, 1, 2, 3], dtype=int32)>, 'y': <tf.Tensor: shape=(4,), dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>},
      dtype=object)

In [50]:
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.hidden = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)
        self.out = tf.keras.layers.Dense(units=10)

    def call(self, inputs):
        x = self.flatten(inputs)
        x = self.hidden(x)
        return self.out(x)

net = MLP()
X = tf.random.uniform((2, 20))
Y = net(X)

In [51]:
net.save_weights('mlp.params')

In [52]:
clone = MLP()
clone.load_weights('mlp.params')

In [53]:
Y_clone = clone(X)
Y_clone == Y

<tf.Tensor: shape=(2, 10), dtype=bool, numpy=
array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True]])>

In [54]:
#5.6

In [55]:
!nvidia-smi

Wed Oct 27 10:36:43 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 455.23.05    CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 3090    On   | 00000000:17:00.0 Off |                  N/A |
|  0%   56C    P8    27W / 370W |   1218MiB / 24268MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 3090    On   | 00000000:65:00.0 Off |                  N/A |
|  0%   

In [56]:
import tensorflow as tf

tf.device('/CPU:0'), tf.device('/GPU:0'), tf.device('/GPU:1')

(<tensorflow.python.eager.context._EagerDeviceContext at 0x7f7ab07a1c10>,
 <tensorflow.python.eager.context._EagerDeviceContext at 0x7f7ab07a1d10>)

In [57]:
len(tf.config.experimental.list_physical_devices('GPU'))

2

In [58]:
def try_gpu(i=0):  #@save
    """如果存在，则返回gpu(i)，否则返回cpu()。"""
    if len(tf.config.experimental.list_physical_devices('GPU')) >= i + 1:
        return tf.device(f'/GPU:{i}')
    return tf.device('/CPU:0')

def try_all_gpus():  #@save
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]。"""
    num_gpus = len(tf.config.experimental.list_physical_devices('GPU'))
    devices = [tf.device(f'/GPU:{i}') for i in range(num_gpus)]
    return devices if devices else [tf.device('/CPU:0')]

try_gpu(), try_gpu(10), try_all_gpus()

(<tensorflow.python.eager.context._EagerDeviceContext at 0x7f7ab07696d0>,
  <tensorflow.python.eager.context._EagerDeviceContext at 0x7f7ab0769890>])

In [59]:
x = tf.constant([1, 2, 3])
x.device

'/job:localhost/replica:0/task:0/device:CPU:0'

In [60]:
with try_gpu():
    X = tf.ones((2, 3))
X

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 1., 1.],
       [1., 1., 1.]], dtype=float32)>

In [61]:
with try_gpu(1):
    Y = tf.random.uniform((2, 3))
Y

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0.08526587, 0.8208443 , 0.93755734],
       [0.89818895, 0.7838323 , 0.13732052]], dtype=float32)>

In [62]:
with try_gpu(1):
    Z = X
print(X)
print(Z)

tf.Tensor(
[[1. 1. 1.]
 [1. 1. 1.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[1. 1. 1.]
 [1. 1. 1.]], shape=(2, 3), dtype=float32)


In [63]:
Y + Z

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1.0852659, 1.8208443, 1.9375573],
       [1.898189 , 1.7838323, 1.1373205]], dtype=float32)>

In [64]:
with try_gpu(1):
    Z2 = Z
Z2 is Z

True

In [65]:
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    net = tf.keras.models.Sequential([
        tf.keras.layers.Dense(1)])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


In [66]:
net(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.2582593],
       [0.2582593]], dtype=float32)>

In [67]:
net.layers[0].weights[0].device, net.layers[0].weights[1].device

('/job:localhost/replica:0/task:0/device:GPU:0',
 '/job:localhost/replica:0/task:0/device:GPU:0')